In [1]:
import spacy
import json
import os
from nltk.corpus import stopwords, words
import pandas as pd
import gensim
import gensim.corpora as corpora
import re
import contractions
import string

In [48]:
files = []
subdir = tuple(os.walk("../MergedData/merged_movie_jsons/2014"))
for file_name in tuple(subdir)[0][-1]:
    with open(f"{subdir[0][0]}/{file_name}", 'r') as f:
        files.append(json.load(f))

In [49]:
corpus = []
my_pos = set(["ADJ"])
nlp = spacy.load("en_core_web_lg") #python -m spacy download en_core_web_lg


for file in files:
    if file["title"] != "Captain America: The Winter Soldier":
        continue
    content = [review["content"] for review in file["reviews"]]
    for review in content:
        s = nlp(review)
        clean = []
        for word in s:
            if word.pos_ in my_pos:
                clean.append(str(word.lemma_))
        corpus.append(" ".join(clean))
    break

In [50]:
# corpus
# remove non english words again
# remove numbers
# remove words shorted than 3 letters
# remove punctuations
# print(type(stopwords.words("english")))

In [51]:
with open("../stopwords.json", "r") as f:
    common = json.load(f)
    common.extend(stopwords.words("english"))
    common = set(common)


eng_words = set(words.words())

def clean(text: str):
    
    # remove numbers
    text = re.sub(r"\d+", "", text)
    # expand contraction
    text = contractions.fix(text)
    # remove links
    text = re.sub(r"(ftp|http[s]?)://\S+", "", text)
    # remove punctuations
    sub_txt = r"[" + re.escape(string.punctuation) + r"]"
    text = re.sub(sub_txt, "", text)

    text = re.sub("[^a-zA-Z0-9 ]+", " ", text)

    # remove emails
    text = re.sub("[\w\.-]+@[\w\.-]+\.\w+", " ", text)
    
    text = " ".join([word for word in text.split() if word not in common and word in eng_words and len(word) > 2])

     # remove multispaces
    text = re.sub('\s+', " ", text)

    # lemmatizer.lemmatize()
    return text.strip()

In [52]:
proc_data = [clean(movie_revs).split() for movie_revs in corpus]
input_dict = corpora.Dictionary(proc_data)
input_corpus = [input_dict.doc2bow(token, allow_update=True) for token in proc_data]

In [53]:
model = gensim.models.TfidfModel(input_corpus, id2word=input_dict,  normalize=True, dictionary=input_dict, slope=2.5, smartirs="Lfc")
vector = model[input_corpus]

In [54]:
d = {}
for v in vector:
    for id, f in v:
        d[input_dict[id]] = f

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gensim/models/tfidfmodel.py:180: RuntimeWarning: Mean of empty slice.
  return (1 + np.log2(tf)) / (1 + np.log2(tf.mean(axis=0)))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [55]:
df = pd.DataFrame(d.values(), index=d.keys(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)

In [56]:
# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 3,
#                        ):
#     print(df["TF-IDF"].between(0.8 * df["TF-IDF"].median(), 1.05 * df["TF-IDF"].median(), inclusive=True))
    # pass

In [57]:
# kmeans = KMeans(n_clusters=100, init="k-means++", n_init=10000)
# vect = TfidfVectorizer(stop_words="english")
# features = vect.fit_transform([ " ".join(data) for data in proc_data])
# kmeans.fit(features)

In [58]:
# df2 = pd.DataFrame(corpus)
# df2["clusters"] = kmeans.labels_
# df2

In [59]:
# print("Cluster centrojds: \n")
# order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
# terms = vect.get_feature_names_out()
# for i in range (100):
#     print("cluster %d:" % i)
#     for j in order_centroids[i, :10]: #print out 10 feature terms of each cluster
#         print(' %s ' % terms [j])

In [64]:
med = df["TF-IDF"].median()
mean = df["TF-IDF"].mean()
vibes = set()
mean = df["TF-IDF"].quantile(.9)
# d = {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)}
for k, v in d.items():
    if 0.95 *  mean <= v <=  1.05 * mean:
        vibes.add(k)
vibes
# print(file["title"])

{'absolutist',
 'accessible',
 'always',
 'ashamed',
 'captive',
 'catchy',
 'connective',
 'controversial',
 'countless',
 'crisp',
 'crooked',
 'customary',
 'damn',
 'deceptive',
 'delightful',
 'displeased',
 'dissimilar',
 'due',
 'dynamic',
 'elementary',
 'encyclopedic',
 'enthralling',
 'exceptional',
 'flat',
 'fleshed',
 'forward',
 'frenetic',
 'fundamental',
 'goofy',
 'grave',
 'gross',
 'grotesque',
 'hopeless',
 'hostile',
 'immediate',
 'invisible',
 'knowing',
 'later',
 'lovely',
 'magical',
 'massive',
 'meaningless',
 'mighty',
 'mysterious',
 'parallel',
 'past',
 'pointless',
 'polish',
 'predictable',
 'quirky',
 'running',
 'savvy',
 'slow',
 'stationary',
 'tad',
 'terrific',
 'thoroughbred',
 'tiny',
 'traitorous',
 'unhelpful',
 'unspoken',
 'useless',
 'wanton',
 'welcome'}

In [61]:
# cnt = {}
# for sent in corpus:
#     sent = clean(sent)
#     for word in set(sent.split()):
#         cnt[word] = cnt.get(word, 0) + 1

# cnt = {k: v for k, v in sorted(cnt.items(), key=lambda item: item[1], reverse=True)}

# for c in cnt:
#     print(c, cnt[c])